# 웹 크롤링
- 목표 : 웹 크롤링 후 DB 연동까지

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# 1. Chrome 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")  # 창 띄우기
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 2. 사이트 접속
driver.get("https://quotes.toscrape.com")
time.sleep(2)  # 로딩 대기

# 3. "Login" 버튼 클릭
login_link = driver.find_element(By.LINK_TEXT, "Login")
login_link.click()

# 4. 결과 확인용 대기
time.sleep(3)

# driver.quit()

# 웹사이트에서 직접 크롤링 (추천X)
- 실무에서 쓰지않음
- 기업에서 자주 바꿈 => 유지보수 불가능

In [ ]:
import requests
from bs4 import BeautifulSoup

# 1. URL 설정
url = "http://quotes.toscrape.com/page/1/"

# 2. HTTP 요청
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# 3. 명언과 저자 추출
quotes = soup.find_all("div", class_="quote")

for quote in quotes:
    text = quote.find("span", class_="text").get_text()
    author = quote.find("small", class_="author").get_text()
    print(f"{text} — {author}")

# NY Times 웹 크롤링 (API)

In [8]:
import requests
import pandas as pd

API_KEY = 'I7b7XyEIcgfyZmPLz1KTlcZKnHpjzP33' # key 값

url = 'https://api.nytimes.com/svc/mostpopular/v2/viewed/1.json' # 각 사이트마다 다름
params = {
    "api-key" : API_KEY
}

response = requests.get(url, params=params)

data = response.json() # json
df = pd.json_normalize(data['results']) # json 형태를 데이터프레임 형태로 변환
df.head(1)

,uri,url,id,asset_id,source,published_date,updated,section,subsection,nytdsection,...,byline,type,title,abstract,des_facet,org_facet,per_facet,geo_facet,media,eta_id
0,nyt://article/145cfe92-7c5f-5da3-8ecf-6167d2e7...,https://www.nytimes.com/2025/06/17/nyregion/br...,100000010234457,100000010234457,New York Times,2025-06-17,2025-06-17 18:49:31,New York,,new york,...,By Luis Ferré-Sadurní,Article,Brad Lander Is Arrested by ICE Agents at Immig...,"Mr. Lander, the New York City comptroller and ...","[Immigration and Emigration, Elections, Mayors]","[Immigration and Customs Enforcement (US), Off...","[Lander, Brad]",[],"[{'type': 'image', 'subtype': 'photo', 'captio...",0


# 단일 학교 1일 급식 정보 가져오기

In [1]:
import requests
import pandas as pd

EDU_CODE = 'B10'                  # 교육청 코드
SCHOOL_CODE = '7021105'           # 학교 코드
API_KEY = '3c3bbe3ed1194b93a7aa44e8e81adee6'
TARGET_DATE = '20241030' # 수요일

# 학교의 이름 가져오는 함수 
def get_school_name(edu_code, school_code):
    print(f'학교 이름 조회 에정 : 교육청={edu_code}, 학교 코드= {school_code}')
    url = 'https://open.neis.go.kr/hub/schoolInfo'
    params = {
        'KEY' : API_KEY,
        'Type' : 'json',
        'ATPT_OFCDC_SC_CODE' : edu_code,     # 시도교육청코드
        'SD_SCHUL_CODE' : school_code        # 행정표준코드
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"응답코드 : {res.status_code}")
        if res.status_code ==200:
            data = res.json()
            if "schoolInfo" in data:
                # 여기 코드가 핵심
                name = data["schoolInfo"][1]["row"][0]["SCHUL_NM"]    # school_info 접속 => 하나하나 접속해서 원하는 정보로 접근
                print(f"학교명: {name}")
                return name
            else:
                print("해당학교는 존재하지 않음", data)
        else:
            print("응답 실패", res.text)
    except Exception as e:
        print(f"학교명 정보 조회 오류 : {e}")
    return "학교명조회불가"

get_school_name(EDU_CODE, SCHOOL_CODE)

학교 이름 조회 에정 : 교육청=B10, 학교 코드= 7021105
응답코드 : 200
학교명: 서울이문초등학교


'서울이문초등학교'

In [2]:
def get_meal_info(edu_code, school_code):
    print(f"급식 정보 조회 중: 날짜={TARGET_DATE}")
    url = "https://open.neis.go.kr/hub/mealServiceDietInfo"
    params = {
        "KEY": API_KEY,
        "Type": "json",
        "ATPT_OFCDC_SC_CODE": edu_code,
        "SD_SCHUL_CODE": school_code,
        "MLSV_YMD": TARGET_DATE
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"mealService 응답코드: {res.status_code}")
        if res.status_code == 200:
            data = res.json()
            if "mealServiceDietInfo" in data:
                print(f"급식 데이터 수: {len(data['mealServiceDietInfo'][1]['row'])}")
                return data["mealServiceDietInfo"][1]["row"]
            else:
                print("mealServiceDietInfo 키 없음:", data)
        else:
            print("응답 실패:", res.text)
    except Exception as e:
        print(f"급식 정보 조회 오류: {e}")
    return []

# 데이터 수집
school_name = get_school_name(EDU_CODE, SCHOOL_CODE)
meals = get_meal_info(EDU_CODE, SCHOOL_CODE)

data = []
for meal in meals:
    data.append({
        "교육청코드": EDU_CODE,
        "학교코드": SCHOOL_CODE,
        "학교명": school_name,
        "급식일자": meal.get("MLSV_YMD"),
        "식사명": meal.get("MMEAL_SC_NM"),
        "급식식단": meal.get("DDISH_NM")
    })

df = pd.DataFrame(data)
print("수집된 행 수:", len(df))
print(df.head())

df.to_csv("서울_초등학교_급식_20241030.csv", index=False, encoding="utf-8-sig")
print("CSV 저장 완료: 서울_초등학교_급식_20241030.csv")

학교 이름 조회 에정 : 교육청=B10, 학교 코드= 7021105
응답코드 : 200
학교명: 서울이문초등학교
급식 정보 조회 중: 날짜=20241030
mealService 응답코드: 200
급식 데이터 수: 1
수집된 행 수: 1
  교육청코드     학교코드       학교명      급식일자 식사명  \
0   B10  7021105  서울이문초등학교  20241030  중식   

                                                급식식단  
0  해물죽 (9.13.17.18)<br/>로제스파게티(우유) (1.2.5.6.12.13...  
CSV 저장 완료: 서울_초등학교_급식_20241030.csv


In [ ]:
import requests
import pandas as pd

EDU_CODE = 'B10'                  # 교육청 코드
SCHOOL_CODE = '7021105'           # 학교 코드
API_KEY = '3c3bbe3ed1194b93a7aa44e8e81adee6'
TARGET_DATE = '20241030' # 수요일

# 학교의 이름 가져오는 함수 
def get_school_name(edu_code, school_code):
    print(f'학교 이름 조회 에정 : 교육청={edu_code}, 학교 코드= {school_code}')
    url = 'https://open.neis.go.kr/hub/schoolInfo'
    params = {
        'KEY' : API_KEY,
        'Type' : 'json',
        'ATPT_OFCDC_SC_CODE' : edu_code,     # 시도교육청코드
        'SD_SCHUL_CODE' : school_code        # 행정표준코드
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"응답코드 : {res.status_code}")
        if res.status_code ==200:
            data = res.json()
            if "schoolInfo" in data:
                # 여기 코드가 핵심
                name = data["schoolInfo"][1]["row"][0]["SCHUL_NM"]    # school_info 접속 => 하나하나 접속해서 원하는 정보로 접근
                print(f"학교명: {name}")
                return name
            else:
                print("해당학교는 존재하지 않음", data)
        else:
            print("응답 실패", res.text)
    except Exception as e:
        print(f"학교명 정보 조회 오류 : {e}")
    return "학교명조회불가"

def get_meal_info(edu_code, school_code):
    print(f"급식 정보 조회 중: 날짜={TARGET_DATE}")
    url = "https://open.neis.go.kr/hub/mealServiceDietInfo"
    params = {
        "KEY": API_KEY,
        "Type": "json",
        "ATPT_OFCDC_SC_CODE": edu_code,
        "SD_SCHUL_CODE": school_code,
        "MLSV_YMD": TARGET_DATE
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"mealService 응답코드: {res.status_code}")
        if res.status_code == 200:
            data = res.json()
            if "mealServiceDietInfo" in data:
                print(f"급식 데이터 수: {len(data['mealServiceDietInfo'][1]['row'])}")
                return data["mealServiceDietInfo"][1]["row"]
            else:
                print("mealServiceDietInfo 키 없음:", data)
        else:
            print("응답 실패:", res.text)
    except Exception as e:
        print(f"급식 정보 조회 오류: {e}")
    return []

# 데이터 수집
school_name = get_school_name(EDU_CODE, SCHOOL_CODE)
meals = get_meal_info(EDU_CODE, SCHOOL_CODE)

data = []
for meal in meals:
    data.append({
        "교육청코드": EDU_CODE,
        "학교코드": SCHOOL_CODE,
        "학교명": school_name,
        "급식일자": meal.get("MLSV_YMD"),
        "식사명": meal.get("MMEAL_SC_NM"),
        "급식식단": meal.get("DDISH_NM")
    })

df = pd.DataFrame(data)
print("수집된 행 수:", len(df))
print(df.head())

df.to_csv("서울_초등학교_급식_20241030.csv", index=False, encoding="utf-8-sig")
print("CSV 저장 완료: 서울_초등학교_급식_20241030.csv")

# 1개의 학교의 한달 데이터 수집
- 날짜만 변경

In [3]:
from datetime import datetime, timedelta
today = datetime.today()
one_month_age = today - timedelta(days=30)
print(today)
print(one_month_age)

FROM_DATE = one_month_age.strftime('%Y%m%d')
TO_DATE = today.strftime('%Y%m%d')

FROM_DATE, TO_DATE

2025-06-18 12:48:16.494880
2025-05-19 12:48:16.494880


('20250519', '20250618')

In [4]:
import requests
import pandas as pd
from datetime import datetime, timedelta

EDU_CODE = 'B10'                  # 교육청 코드
SCHOOL_CODE = '7021105'           # 학교 코드
API_KEY = '3c3bbe3ed1194b93a7aa44e8e81adee6'

# 최근 한달 기간 설정
today = datetime.today()
one_month_age = today - timedelta(days=30)
FROM_DATE = one_month_age.strftime('%Y%m%d')
TO_DATE = today.strftime('%Y%m%d')

# 학교의 이름 가져오는 함수 
def get_school_name(edu_code, school_code):
    print(f'학교 이름 조회 에정 : 교육청={edu_code}, 학교 코드= {school_code}')
    url = 'https://open.neis.go.kr/hub/schoolInfo'
    params = {
        'KEY' : API_KEY,
        'Type' : 'json',
        'ATPT_OFCDC_SC_CODE' : edu_code,     # 시도교육청코드
        'SD_SCHUL_CODE' : school_code       # 행정표준코드
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"응답코드 : {res.status_code}")
        if res.status_code == 200:
            data = res.json()
            if "schoolInfo" in data:
                # 여기 코드가 핵심
                name = data["schoolInfo"][1]["row"][0]["SCHUL_NM"]    # school_info 접속 => 하나하나 접속해서 원하는 정보로 접근
                print(f"학교명: {name}")
                return name
            else:
                print("해당학교는 존재하지 않음", data)
        else:
            print("응답 실패", res.text)
    except Exception as e:
        print(f"학교명 정보 조회 오류 : {e}")
    return "학교명조회불가"

def get_meal_info(edu_code, school_code, from_date, to_date):
    print(f"급식 정보 조회 중: 날짜={TARGET_DATE}")
    url = "https://open.neis.go.kr/hub/mealServiceDietInfo"
    params = {
        "KEY": API_KEY,
        "Type": "json",
        "ATPT_OFCDC_SC_CODE": edu_code,
        "MLSV_FROM_YMD": from_date,
        "MLSV_TO_YMD": to_date,
        "pIndex" : 1,
        "pSize" : 100
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"mealService 응답코드: {res.status_code}")
        if res.status_code == 200:
            data = res.json()
            if "mealServiceDietInfo" in data:
                print(f"급식 데이터 수: {len(data['mealServiceDietInfo'][1]['row'])}")
                return data["mealServiceDietInfo"][1]["row"]
            else:
                print("mealServiceDietInfo 키 없음:", data)
        else:
            print("응답 실패:", res.text)
    except Exception as e:
        print(f"급식 정보 조회 오류: {e}")
    return []

# 데이터 수집
school_name = get_school_name(EDU_CODE, SCHOOL_CODE)
meals = get_meal_info(EDU_CODE, SCHOOL_CODE, FROM_DATE, TO_DATE)

data = []
for meal in meals:
    data.append({
        "교육청코드": EDU_CODE,
        "학교코드": SCHOOL_CODE,
        "학교명": school_name,
        "급식일자": meal.get("MLSV_YMD"),
        "식사명": meal.get("MMEAL_SC_NM"),
        "급식식단": meal.get("DDISH_NM")
    })

df = pd.DataFrame(data)
print("수집된 행 수:", len(df))
print(df.head())

#df.to_csv("서울_초등학교_급식_20241030.csv", index=False, encoding="utf-8-sig")
#print("CSV 저장 완료: 서울_초등학교_급식_20241030.csv")

학교 이름 조회 에정 : 교육청=B10, 학교 코드= 7021105
응답코드 : 200
학교명: 서울이문초등학교
급식 정보 조회 중: 날짜=20241030
mealService 응답코드: 200
mealServiceDietInfo 키 없음: {'RESULT': {'CODE': 'ERROR-300', 'MESSAGE': '필수 값이 누락되어 있습니다. 요청인자를 참고 하십시오.'}}
수집된 행 수: 0
Empty DataFrame
Columns: []
Index: []


# 서울에 있는 모든 고등학교 최근 1개월 데이터 수정
- 가장 중요한 건, 학교명과 학교코드가 필요

In [6]:
import requests
import pandas as pd
from datetime import datetime, timedelta

EDU_CODE = 'B10'                  # 교육청 코드
SCHOOL_CODE = '7021105'           # 학교 코드
API_KEY = '3c3bbe3ed1194b93a7aa44e8e81adee6'


# 최근 한달 기간 설정
today = datetime.today()
one_month_ago = today - timedelta(days=30)
FROM_DATE = one_month_ago.strftime('%Y%m%d')
TO_DATE = today.strftime('%Y%m%d')

def get_meal_info(edu_code, school_code, from_date, to_date):
    print(f"급식 정보 조회 중: 날짜={TARGET_DATE}")
    url = "https://open.neis.go.kr/hub/mealServiceDietInfo"
    params = {
        "KEY": API_KEY,
        "Type": "json",
        "ATPT_OFCDC_SC_CODE": edu_code,
        "SD_SCHUL_CODE": school_code,
        "MLSV_FROM_YMD": from_date, 
        "MLSV_TO_YMD" : to_date, 
        "pIndex" : 1, 
        "pSize" : 100
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"mealService 응답코드: {res.status_code}")
        if res.status_code == 200:
            data = res.json()
            if "mealServiceDietInfo" in data:
                print(f"급식 데이터 수: {len(data['mealServiceDietInfo'][1]['row'])}")
                return data["mealServiceDietInfo"][1]["row"]
            else:
                print("mealServiceDietInfo 키 없음:", data)
        else:
            print("응답 실패:", res.text)
    except Exception as e:
        print(f"급식 정보 조회 오류: {e}")
    return []

all_data = []
for school_code, school_name in tqdm(highschool_list, desc="고등학교 급식 조회"):
    print(school_code, school_name)
    meals = get_meal_info(EDU_CODE, school_code, FROM_DATE, TO_DATE)
    for meal in meals:
        all_data.append({
            "교육청코드": EDU_CODE,
            "학교코드": school_code,
            "학교명": school_name,
            "급식일자": meal.get("MLSV_YMD"),
            "식사명": meal.get("MMEAL_SC_NM"),
            "급식식단": meal.get("DDISH_NM")
        })
    time.sleep(0.1)

# 저장
df = pd.DataFrame(all_data)
print("수집된 급식 행 수:", len(df))
print(df.head())

NameError: name 'tqdm' is not defined

In [ ]:
df.to_csv("서울_고등학교_급식정보.csv", index=False, encoding="utf-8-sig")
print("CSV 저장 완료: 서울_고등학교_급식정보_20241030.csv")

# 전국시도교육청 기반으로 고등학교 전체 급식데이터 수집
ChatGPt 사용하지 않고 진행 (반복문~반복문~조건문, 기초문법 확장한 개념)

In [ ]:
EDU_CODES = [
    "B10", "C10", "D10", "E10", "F10", "G10", "H10", "I10", "J10",
    "K10", "M10", "N10", "P10", "Q10", "R10", "S10", "T10"
]

import requests
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import time

EDU_CODE = 'B10'
API_KEY = '3c3bbe3ed1194b93a7aa44e8e81adee6'


# 최근 한달 기간 설정
today = datetime.today()
one_month_ago = today - timedelta(days=30)
FROM_DATE = one_month_ago.strftime('%Y%m%d')
TO_DATE = today.strftime('%Y%m%d')

# 고등학교만 조회
def get_highschools(edu_code):
    url = 'https://open.neis.go.kr/hub/schoolInfo'
    params = {
        'KEY': API_KEY, 
        'Type' : 'json', 
        'ATPT_OFCDC_SC_CODE' : edu_code, 
        'pIndex': 1, 
        'pSize': 1000
    }
    highschools = []
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"응답코드 : {res.status_code}")
        if res.status_code == 200 and 'schoolInfo' in res.json():
            rows = res.json()['schoolInfo'][1]['row']
            for row in rows:
                if row.get('SCHUL_KND_SC_NM') == '고등학교':
                    highschools.append((row['SD_SCHUL_CODE'], row['SCHUL_NM']))
                else:
                    pass
        else:
            print("200은 뜨나, schoolinfo은 없음")
    except Exception as e:
        print("고등학교 목록 오류: {e}")
    return highschools

highschool_list = get_highschools(EDU_CODE)
len(highschool_list)

def get_meal_info(edu_code, school_code, from_date, to_date):
    print(f"급식 정보 조회 중: 날짜={TARGET_DATE}")
    url = "https://open.neis.go.kr/hub/mealServiceDietInfo"
    params = {
        "KEY": API_KEY,
        "Type": "json",
        "ATPT_OFCDC_SC_CODE": edu_code,
        "MLSV_FROM_YMD": from_date,
        "MLSV_TO_YMD": to_date,
        "pIndex" : 1,
        "pSize" : 100
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"mealService 응답코드: {res.status_code}")
        if res.status_code == 200:
            data = res.json()
            if "mealServiceDietInfo" in data:
                print(f"급식 데이터 수: {len(data['mealServiceDietInfo'][1]['row'])}")
                return data["mealServiceDietInfo"][1]["row"]
            else:
                print("mealServiceDietInfo 키 없음:", data)
        else:
            print("응답 실패:", res.text)
    except Exception as e:
        print(f"급식 정보 조회 오류: {e}")
    return []


all_data =[]
for school_code, school_name in tqdm(highschool_list, desc="고등학교 급식 조회"):
    print(school_code, school_name)
    meals = get_meal_info(EDU_CODE, school_code, FROM_DATE, TO_DATE)
    print(meals)
    for meal in meals:
        all_data.append({
            "교육청코드": EDU_CODE,
            "학교코드": school_code,
            "학교명": school_name,
            "급식일자": meal.get("MLSV_YMD"),
            "식사명": meal.get("MMEAL_SC_NM"),
            "급식식단": meal.get("DDISH_NM")
        })
    time.sleep(0.1)

# 저장
df = pd.DataFrame(all_data)
print("수집된 급식 행 수:", len(df))
print(df.head())

In [19]:
EDU_CODES = [
    "B10", "C10", "D10", "E10", "F10", "G10", "H10", "I10", "J10",
    "K10", "M10", "N10", "P10", "Q10", "R10", "S10", "T10"
]
import requests
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import time

API_KEY = '3c3bbe3ed1194b93a7aa44e8e81adee6'

# 최근 한달 기간 설정
today = datetime.today()
one_month_ago = today - timedelta(days=30)
FROM_DATE = one_month_ago.strftime('%Y%m%d')
TO_DATE = today.strftime('%Y%m%d')
                         
def get_highschools(edu_code):
    url = 'https://open.neis.go.kr/hub/schoolInfo'
    params = {
        'KEY': API_KEY, 
        'Type' : 'json', 
        'ATPT_OFCDC_SC_CODE' : edu_code, 
        'pIndex': 1, 
        'pSize': 1000
    }
    highschools = []
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"응답코드 : {res.status_code}")
        if res.status_code == 200 and 'schoolInfo' in res.json():
            rows = res.json()['schoolInfo'][1]['row']
            for row in rows:
                if row.get('SCHUL_KND_SC_NM') == '고등학교':
                    highschools.append((row['SD_SCHUL_CODE'], row['SCHUL_NM']))
                else:
                    pass
        else:
            print("200은 뜨나, schoolinfo은 없음")
    except Exception as e:
        print("고등학교 목록 오류: {e}")
    return highschools

highschool_list = get_highschools(EDU_CODE)
len(highschool_list)


def get_meal_info(edu_code, school_code, from_date, to_date):
    print(f"급식 정보 조회 중: 날짜={TARGET_DATE}")
    url = "https://open.neis.go.kr/hub/mealServiceDietInfo"
    params = {
        "KEY": API_KEY,
        "Type": "json",
        "ATPT_OFCDC_SC_CODE": edu_code,
        "MLSV_FROM_YMD": from_date,
        "MLSV_TO_YMD": to_date,
        "pIndex" : 1,
        "pSize" : 100
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        print(f"mealService 응답코드: {res.status_code}")
        if res.status_code == 200:
            data = res.json()
            if "mealServiceDietInfo" in data:
                print(f"급식 데이터 수: {len(data['mealServiceDietInfo'][1]['row'])}")
                return data["mealServiceDietInfo"][1]["row"]
            else:
                print("mealServiceDietInfo 키 없음:", data)
        else:
            print("응답 실패:", res.text)
    except Exception as e:
        print(f"급식 정보 조회 오류: {e}")
    return []


all_data =[]
for EDU_CODE in EDU_CODES:
    highschool_list = get_highschools(EDU_CODE)
    for school_code, school_name in tqdm(highschool_list, desc="고등학교 급식 조회"):
        print(school_code, school_name)
        meals = get_meal_info(EDU_CODE, school_code, FROM_DATE, TO_DATE)
        print(meals)
        for meal in meals:
            all_data.append({
                "교육청코드": EDU_CODE,
                "학교코드": school_code,
                "학교명": school_name,
                "급식일자": meal.get("MLSV_YMD"),
                "식사명": meal.get("MMEAL_SC_NM"),
                "급식식단": meal.get("DDISH_NM")
            })
        time.sleep(0.1)

# 저장
df = pd.DataFrame(all_data)
print("수집된 급식 행 수:", len(df))
print(df.head())

응답코드 : 200
응답코드 : 200


 ... (more hidden) ...

7010057 가락고등학교
급식 정보 조회 중: 날짜=20241030
mealService 응답코드: 200
mealServiceDietInfo 키 없음: {'RESULT': {'CODE': 'ERROR-300', 'MESSAGE': '필수 값이 누락되어 있습니다. 요청인자를 참고 하십시오.'}}
[]
7011169 가재울고등학교
급식 정보 조회 중: 날짜=20241030


 ... (more hidden) ...

mealService 응답코드: 200
mealServiceDietInfo 키 없음: {'RESULT': {'CODE': 'ERROR-300', 'MESSAGE': '필수 값이 누락되어 있습니다. 요청인자를 참고 하십시오.'}}
[]
7010117 강동고등학교
급식 정보 조회 중: 날짜=20241030
mealService 응답코드: 200
mealServiceDietInfo 키 없음: {'RESULT': {'CODE': 'ERROR-300', 'MESSAGE': '필수 값이 누락되어 있습니다. 요청인자를 참고 하십시오.'}}
[]


...

9290055 한림고등학교
급식 정보 조회 중: 날짜=20241030
mealService 응답코드: 200
mealServiceDietInfo 키 없음: {'RESULT': {'CODE': 'ERROR-300', 'MESSAGE': '필수 값이 누락되어 있습니다. 요청인자를 참고 하십시오.'}}
[]
9290054 한림공업고등학교
급식 정보 조회 중: 날짜=20241030


 ... (more hidden) ...

mealService 응답코드: 200
mealServiceDietInfo 키 없음: {'RESULT': {'CODE': 'ERROR-300', 'MESSAGE': '필수 값이 누락되어 있습니다. 요청인자를 참고 하십시오.'}}
[]
9290103 함덕고등학교
급식 정보 조회 중: 날짜=20241030
mealService 응답코드: 200
mealServiceDietInfo 키 없음: {'RESULT': {'CODE': 'ERROR-300', 'MESSAGE': '필수 값이 누락되어 있습니다. 요청인자를 참고 하십시오.'}}
[]


 ... (more hidden) ...

수집된 급식 행 수: 0
Empty DataFrame
Columns: []
Index: []


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import time

EDU_CODES = [
    "B10", "C10", "D10", "E10", "F10", "G10", "H10", "I10", "J10",
    "K10", "M10", "N10", "P10", "Q10", "R10", "S10", "T10"
]

API_KEY = '3c3bbe3ed1194b93a7aa44e8e81adee6'

today = datetime.today()
one_month_ago = today - timedelta(days=30)
FROM_DATE = one_month_ago.strftime('%Y%m%d')
TO_DATE = today.strftime('%Y%m%d')


def get_highschools(edu_code):
    url = 'https://open.neis.go.kr/hub/schoolInfo'
    params = {
        'KEY': API_KEY, 
        'Type': 'json', 
        'ATPT_OFCDC_SC_CODE': edu_code, 
        'pIndex': 1, 
        'pSize': 1000
    }
    highschools = []
    try:
        res = requests.get(url, params=params, timeout=5)
        if res.status_code == 200 and 'schoolInfo' in res.json():
            rows = res.json()['schoolInfo'][1]['row']
            for row in rows:
                if row.get('SCHUL_KND_SC_NM') == '고등학교':
                    highschools.append((row['SD_SCHUL_CODE'], row['SCHUL_NM']))
    except Exception as e:
        print(f"[오류] 고등학교 목록 조회 실패: {e}")
    return highschools


def get_meal_info(edu_code, school_code, from_date, to_date):
    url = "https://open.neis.go.kr/hub/mealServiceDietInfo"
    params = {
        "KEY": API_KEY,
        "Type": "json",
        "ATPT_OFCDC_SC_CODE": edu_code,
        "SD_SCHUL_CODE": school_code,
        "MLSV_FROM_YMD": from_date,
        "MLSV_TO_YMD": to_date,
        "pIndex": 1,
        "pSize": 100
    }
    try:
        res = requests.get(url, params=params, timeout=5)
        if res.status_code == 200:
            data = res.json()
            if "mealServiceDietInfo" in data:
                return data["mealServiceDietInfo"][1]["row"]
    except Exception as e:
        print(f"[오류] 급식 정보 조회 실패 ({school_code}): {e}")
    return []


all_data = []

for EDU_CODE in EDU_CODES:
    highschool_list = get_highschools(EDU_CODE)
    for school_code, school_name in tqdm(highschool_list, desc=f"[{EDU_CODE}] 고등학교 급식 조회"):
        meals = get_meal_info(EDU_CODE, school_code, FROM_DATE, TO_DATE)
        for meal in meals:
            all_data.append({
                "교육청코드": EDU_CODE,
                "학교코드": school_code,
                "학교명": school_name,
                "급식일자": meal.get("MLSV_YMD"),
                "식사명": meal.get("MMEAL_SC_NM"),
                "급식식단": meal.get("DDISH_NM")
            })
        time.sleep(0.1)

# 결과 저장
df = pd.DataFrame(all_data)
print("총 수집된 급식 정보:", len(df))
# print(df.head())
df.head()